# Evolution of urban patterns: urban morphology as an open reproducible data science

## Urban morphometrics

This is the first notebook in a sequence of three. The notebook downloads data from OpenStreetMap, generates morphological tessellation and compute a set of morphometric characters.

It requires `data/case_studies.csv` input with origins of case studies.

Date: February 5, 2021

---

We start with the import of libraries used in this notebook.

In [1]:
import pandas as pd
import geopandas as gpd
import momepy as mm
import pyproj
import osmnx
import libpysal
from shapely.geometry import box
from time import time

We will use `osmnx` to download data and specify the timestamp `2021-02-05` to ensure the script always downloads the same data from overpass.

In [2]:
osmnx.config(overpass_settings='[out:json][timeout:90][date:"2021-02-05T00:00:00Z"]')

The only input of the whole analysis is the CSV with case studies.

In [3]:
cases = pd.read_csv("data/case_studies3.csv")
cases

,case,period,origin
0,Connaught Place,pre-industrial,"(77.2194442012068, 28.63264672036609)"
1,Nehru Place,pre-industrial,"(77.25246542282517, 28.54930101761747)"
2,Baner gaon,pre-industrial,"(73.78743391399972, 18.559120436182322)"
3,Siripuram,pre-industrial,"(83.31672292288425, 17.719918874948867)"
4,Manikonda,pre-industrial,"(78.37094172770058, 17.411693315316967)"
5,T Nagar,pre-industrial,"(80.23817967215369, 13.043225604625261)"
6,Cuffe Parade,pre-industrial,"(72.82000253375675, 18.91442395337802)"
7,Ballard Estate,pre-industrial,"(72.83688268053827, 18.936504427534746)"
8,Indiranagar,industrial,"(77.64059073182727, 12.975796346358312)"
9,Electronic City,industrial,"(77.67473906976637, 12.836071363558153)"


The following cell loops over all cases in `cases` DataFrame and does the sequence of the following steps:

1. Download building footprints within 800 m radius around the origin of the case study.
2. Reproject and preprocess them in a way which results in a GeoDataFrame of Polygon geometries.
3. Download street network within 1000 m radius around the origin of the case study.
4. Reproject and convert street network from graph to GeoDataFrame
5. Generate [morphological tessellation](http://docs.momepy.org/en/stable/user_guide/elements/tessellation.html) based on building footprints.
6. Measure a selection of morphometric characters either using GeoPandas or momepy.
7. Link all characters to morphological tessellation cells.
8. Save the result to GeoPackage.

In [4]:
%%capture --no-stdout

for i, coords in cases.origin.iteritems():
    s = time()
    """
    donwload and process elements of urban form
    """
    # download buildings from OSM
    point = tuple(map(float, coords[1:-1].split(', ')))[::-1]
    buildings = osmnx.geometries.geometries_from_point(point, dist=800, tags={'building':True})
    buildings = osmnx.projection.project_gdf(buildings)
    buildings = buildings[buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]
    buildings = buildings.reset_index(drop=True).explode().reset_index(drop=True)
    buildings = buildings[["geometry"]]

    # download streets from OSM
    streets_graph = osmnx.graph_from_point(point, 1000, network_type='drive')
    streets_graph = osmnx.get_undirected(streets_graph)
    streets_graph = osmnx.projection.project_graph(streets_graph)
    streets = osmnx.graph_to_gdfs(streets_graph, nodes=False, edges=True,
                                            node_geometry=False, fill_edge_geometry=True)
    streets = streets[["geometry"]]

    # check CRS
    assert buildings.crs.equals(streets.crs)

    # generate tessellation
    buildings['uID'] = range(len(buildings))
    tessellation = mm.Tessellation(buildings, "uID", limit=box(*buildings.total_bounds), verbose=False).tessellation

    """
    measure morphometric characters
    """
    # cell area
    tessellation['cell_area'] = tessellation.area

    # building area
    buildings['blg_area'] = buildings.area

    # coverage area ratio
    tessellation["car"] = mm.AreaRatio(tessellation, buildings, "cell_area", buildings.area, "uID").series

    # segment length
    streets["length"] = streets.length

    # segment linearity
    streets["linearity"] = mm.Linearity(streets, verbose=False).series

    # street profile
    profile = mm.StreetProfile(streets, buildings, distance=3)
    streets["width"] = profile.w
    streets["width_deviation"] = profile.wd
    streets["openness"] = profile.o

    # perimeter wall
    buildings["wall"] = mm.PerimeterWall(buildings, verbose=False).series

    # generate binary contiguity-based W objects of first order and inclusive order 3
    W1 = libpysal.weights.Queen.from_dataframe(tessellation, ids="uID")
    W3 = mm.sw_high(k=3, weights=W1)

    # building adjacency
    buildings["adjacency"] = mm.BuildingAdjacency(buildings, W3, "uID", verbose=False).series

    # interbuilding distance
    buildings['neighbour_distance'] = mm.NeighborDistance(buildings, W1, 'uID', verbose=False).series

    # generate networkx.MultiGraph object
    G = mm.gdf_to_nx(streets)

    # meshedness
    G = mm.meshedness(G, verbose=False)

    # convert networkx.MultiGraph to geopandas.GeoDataFrames
    nodes, edges = mm.nx_to_gdf(G)

    """
    link all characters to tessellation cells
    """
    # merge street network edges based on proximity
    edges["nID"] = range(len(edges))
    buildings["nID"] = mm.get_network_id(buildings, edges, "nID", 500, verbose=False)
    buildings = buildings.merge(edges.drop(columns="geometry"), on="nID", how="left")

    # merge nodes based on edge ID and proximity
    buildings["nodeID"] = mm.get_node_id(buildings, nodes, edges, "nodeID", "nID", verbose=False)
    buildings = buildings.merge(nodes.drop(columns="geometry"), on="nodeID", how="left")

    # merge buildings based on a shared attribute
    tessellation = tessellation.merge(buildings.drop(columns="geometry"), on="uID", how="left")

    """
    save all to GeoPackage
    """
    tessellation.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="tessellation", driver="GPKG")
    buildings.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="buildings", driver="GPKG")
    edges.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="edges", driver="GPKG")
    nodes.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="nodes", driver="GPKG")

    print(f"{cases.loc[i, 'case']} done in {time() - s} seconds.")

Connaught Place done in 34.291189432144165 seconds.
Nehru Place done in 69.05928802490234 seconds.
Baner gaon done in 43.451839447021484 seconds.
Siripuram done in 54.67616844177246 seconds.
Manikonda done in 55.086590051651 seconds.
T Nagar done in 125.48621535301208 seconds.
Cuffe Parade done in 32.0427827835083 seconds.
Ballard Estate done in 91.3754050731659 seconds.
Indiranagar done in 189.21056866645813 seconds.
Electronic City done in 28.166919708251953 seconds.
Nariman Point done in 37.316638469696045 seconds.
Bandra Kurla Complex done in 32.64209985733032 seconds.
Koramangala done in 147.9570643901825 seconds.
Bardi done in 27.687819004058838 seconds.
Gachibowli done in 45.832696199417114 seconds.
Abids done in 109.9830949306488 seconds.
Gujarat International Finance Tec-City done in 14.547625303268433 seconds.
Sector 17 done in 42.23656725883484 seconds.
Gandhipuram done in 25.208107948303223 seconds.
Nagar Road done in 134.16915678977966 seconds.
Viman nagar done in 57.62357

In [6]:
# Alert
i=0
while i < 5:
    # Play sound when done
    from playsound import playsound
    playsound("C:/Users/HP/Downloads/beep.mp3")
    i=i+2